In [1]:
from dataloader import CustomDataLoader

In [450]:
data = CustomDataLoader('./train/', num_news=10)

txt파일로 출력됐습니다~!


100%|██████████| 2/2 [00:01<00:00,  1.99it/s]

토큰화된 데이터셋도 출력됐슴둥~!
['보수', '진영', '사분', '오열', '속', '‘', '국민', '통합', '연대', '’', '띄운', '비', '박계', '크리스마스', '를', '앞둔', '지난', '23일', '오전', '서울', '프레', '스', '센터', '국제', '회의실', '.', '\n  ', '보수', '분열', '극복', '을', '내', '건', '‘', '국민', '통합', '연대', '’', '가', '창립', '대회', '를', '열었다', '.', '\n  ', '햇살', '없이', '착', '가라앉은', '날씨', '에', '동지', '바람', '이', '매서웠', '지만', '행사', '장', '안은', '달아올랐다', '.', '\n  ', '문재인', '정권', '을', '향', '한', '맹폭격', '이', '이어졌고', '‘', '무능', ',', '기만', '의', '오만', '방자', '한', '정권', '에', '사망', '을', '선고', '한다', '’', '는', '창립', '선언문', '이', '나왔다', '.', '\n  ', '홍준표', '전', '자유', '한국', '당', '대표', '등', '전', '현직', '의원', '20', '여명', '을', '포함', '해', '500', '여명', '이', '자리', '를', '빼곡', '하', '게', '메웠다', '.', '\n  ', '총선', '이', '불과', '석', '달', '남짓', '이다', '.', '\n  ', '야권', '인사', '들', '이', '정권', '을', '두', '들', '겨', '패', '는', '거야', '이상한', '일이', '아니다', '.', '\n  ', '눈', '여겨', '볼', '대목', '은', '모인', '사람', '이', '대부분', '친', '이', '·', '비', '박계', '(', '친', '이명박', '·', '비', '박근혜', ')', '인사', '들', '이란', '점',

In [451]:
import torch
import torch.nn as nn
import numpy as np

In [452]:
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        self.emb = nn.Embedding(n_class, embedding_dim=64)
        self.enc_cell = nn.RNN(input_size=64, hidden_size=n_hidden, dropout=0.5) # 인코딩
        self.dec_cell = nn.RNN(input_size=64, hidden_size=n_hidden, dropout=0.5) # 디코딩
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = self.emb(enc_input)
        dec_input = self.emb(dec_input)
        enc_input = enc_input.transpose(0, 1) # enc_input: [input_size, batch_size, embedding_size]
        dec_input = dec_input.transpose(0, 1) # dec_input: [input_size, batch_size, embedding_size]

        # enc_states : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        _, enc_states = self.enc_cell(enc_input, enc_hidden)

        # outputs : [max_len+1(=6), batch_size, num_directions(=1) * n_hidden(=128)]
        outputs, _ = self.dec_cell(dec_input, enc_states)

        _outputs = outputs.transpose(0, 1) # _outputs (batch_size, input_size(1(E) + 최대 글자 수), n_hidden)
        pred = self.fc(_outputs) # pred : [batch_size, max_len+1(=6), n_class]
        return pred

    def predict(self, input_seq, max_len=50, end_token=2):
        # 1. 인코딩
        input_seq = input_seq.to(device)
        input_embbeded = self.emb(input_seq).transpose(0, 1)
        batch_size = input_seq.shape[0]  # 배치 크기 얻기
        hidden_state_enc = torch.zeros(1, batch_size, n_hidden).to(device)
        _, encoder_hidden = self.enc_cell(input_embbeded, hidden_state_enc)
    
        # 2. 디코딩 시작 토큰 설정
        start_token = torch.ones((batch_size,), dtype=torch.long).to(device)
        decoder_input = start_token

        # 예측된 시퀀스 저장
        predicted_seq = []

        for _ in range(max_len):
            decoder_input_embbeded = self.emb(decoder_input).unsqueeze(0)
            decoder_output, encoder_hidden = self.dec_cell(decoder_input_embbeded, encoder_hidden)

            # linear layer를 통과하여 예측 결과 얻기
            prediction = self.fc(decoder_output.squeeze(0))

            # 가장 확률이 높은 토큰 선택
            predicted_token = prediction.argmax(dim=-1)

            predicted_seq.append(predicted_token.cpu().numpy())

            # 종료 토큰 체크
            if (predicted_token == end_token).any().item():
                break

            # 다음 입력값으로 사용
            decoder_input = predicted_token

        return np.array(predicted_seq).T


In [453]:
torch.zeros(10).long()

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [454]:
n_hidden = 128
n_class = len(data.vocab_dict) + 4

In [455]:
from torch.utils.data import Dataset
class S2SDataset(Dataset):

    def __init__(self, seq_data):
        super().__init__()
        self.feature_in = torch.LongTensor(seq_data[0])
        self.feature_out = torch.LongTensor(seq_data[1])
        self.target = torch.LongTensor(seq_data[2])




    def __len__(self):
        return self.feature_in.shape[0]

    def __getitem__(self, idx):
        return self.feature_in[idx], self.feature_out[idx], self.target[idx]

In [456]:
dataset = S2SDataset(data.dataset)

In [457]:
from torch.utils.data import DataLoader
dl = DataLoader(dataset, batch_size=10, shuffle=False)

In [458]:
nn.Embedding(n_class, embedding_dim=64)(next(iter(dl))[0]).shape

torch.Size([10, 552, 64])

In [459]:
nn.Embedding(n_class, embedding_dim=64)(next(iter(dl))[1][0][0]).shape

torch.Size([64])

In [460]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [475]:
model = Seq2Seq()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [476]:
from tqdm import tqdm

In [483]:
loss_list = []
for epoch in tqdm(range(1000)):
    hidden = torch.zeros(1, 10, 128).to(device)
    loss = 0
    for input, output, target in dl:
        input, output, target = input.to(device), output.to(device), target.to(device)
        output = model(input, hidden, output)
        for i in  range(0, len(target)):
            loss += criterion(output[i], target[i])

    loss /= dl.batch_size
    loss /= len(dl)
    loss_list.append(loss)
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

 10%|█         | 101/1000 [00:12<02:03,  7.31it/s]

Epoch: 0100 cost = 0.049942


 20%|██        | 200/1000 [00:27<02:20,  5.70it/s]

Epoch: 0200 cost = 0.049763


 30%|███       | 300/1000 [00:45<02:04,  5.62it/s]

...

 90%|████████▉ | 899/1000 [03:33<00:47,  2.14it/s]

Epoch: 0900 cost = 0.049521


100%|█████████▉| 999/1000 [04:20<00:00,  2.37it/s]

Epoch: 1000 cost = 0.049493


100%|██████████| 1000/1000 [04:20<00:00,  3.84it/s]


In [484]:
# best_model_path = './news_sum_model.pt'
# model = torch.load(best_model_path, map_location='cpu')

In [485]:
next(iter(dl))[0][0].shape

torch.Size([552])

In [486]:
nn.Embedding(n_class, embedding_dim=64)(next(iter(dl))[0]).shape

torch.Size([10, 552, 64])

In [487]:
result = model.predict(next(iter(dl))[0])

In [488]:
list(result)

[array([  10,   11,  270, 1699,  270,   37,   38,   45,   75,   76,   77,
         164,   74,   78,   79,   81,   82, 1543,  562,   33,   84,   58,
          86], dtype=int64),
 array([  10,   11,  270, 1699,  270,   37,   38,   45,   75,   76,   77,
         164,   74,   78,   79,   81,   82, 1543,  562,   33,   84,   58,
          86], dtype=int64),
 array([  10,   11,  270, 1699,  270,   37,   38,   45,   75,   76,   77,
         164,   74,   78,   79,   81,   82, 1543,  562,   33,   84,   58,
          86], dtype=int64),
 array([  10,   11,  270, 1699,  270,   37,   38,   45,   75,   76,   77,
         164,   74,   78,   79,   81,   82, 1543,  562,   33,   84,   58,
          86], dtype=int64),
 array([  10,   11,  270, 1699,  270,   37,   38,   45,   75,   76,   77,
         164,   74,   78,   79,   81,   82, 1543,  562,   33,   84,   58,
          86], dtype=int64),
 array([  10,   11,  270, 1699,  270,   37,   38,   45,   75,   76,   77,
         164,   74,   78,   79,   81,   8

In [489]:
for i in result[0]:
    print(i)

10
11
270
1699
270
37
38
45
75
76
77
164
74
78
79
81
82
1543
562
33
84
58
86


In [490]:
prediction = ""
for i in result[0]:
    prediction += data.num_to_vocab[i]
prediction

'국민통합연대가연창립대회에자유한국당홍전대표등의원20여명을포함한500'

In [491]:
prediction = ""
for i in result[1]:
    prediction += data.num_to_vocab[i]
prediction

'국민통합연대가연창립대회에자유한국당홍전대표등의원20여명을포함한500'

In [492]:
prediction = ""
for i in result[2]:
    prediction += data.num_to_vocab[i]
prediction

'국민통합연대가연창립대회에자유한국당홍전대표등의원20여명을포함한500'

In [493]:
prediction = ""
for i in result[3]:
    prediction += data.num_to_vocab[i]
prediction

'국민통합연대가연창립대회에자유한국당홍전대표등의원20여명을포함한500'

In [494]:
prediction = ""
for i in result[4]:
    prediction += data.num_to_vocab[i]
prediction

'국민통합연대가연창립대회에자유한국당홍전대표등의원20여명을포함한500'

In [495]:
data.df_result

,passage,summary
0,보수진영 사분오열 속 ‘국민통합연대’ 띄운 비박계 크리스마스를 앞둔 지난 23일 오...,국민통합연대가 연 창립대회에 자유한국당 홍 전 대표 등 의원 20여 명을 포함한 5...
1,가난 속에서 맨손으로 혼자 창업해 30대 중반에 코스닥 상장까지 일궈낸 이가 있다....,아이엘사이언스 대표 송 씨는 발광다이오드 조명용 실리콘 렌즈를 세계 최초로 개발하여...
2,"SK텔레콤은 ‘T끼리 온가족 할인’ 요금제로, 가족 구성원의 가입 합산 기간이...",SK텔레콤은 T끼리 온가족 할인으로 요금을 깎아주거나 30년 이상 가입자를 위해 호...
3,“박스 테이프는 어디 갔죠?” 1일 오전 서울 중구의 한 대형마트.\n 장을 본 ...,대형마트 3사가 장바구니 사용을 독려하고 플라스틱 폐기물을 줄이기 위해 포장용 테이...
4,"현대차그룹 고급 브랜드 제네시스의 첫 스포츠유틸리티차량(SUV), GV80이 드디어...",현대차그룹은 제네시스를 이끌 GV80의 내·외관 사진을 공개하며 이달 중 출시한다고...
5,회사원 주동형(30) 씨는 식당에서 밥값을 낼 때 신용카드 대신 휴대전화를 꺼낸다....,금융감독원에 따르면 2017년~2018년 오프라인 간편결제 이용금액은 19조 500...
6,이낙연 국무총리가 후임 정세균 총리 후보자의 국회 인준과 무관하게 16일 직전 총리...,문 대통령은 이 총리가 국무회의가 열리는 14일이나 15일 당으로 돌아가는 것을 승...
7,지난해 12월 30일 오후 10시 서울 송파동 방이삼거리 서쪽 방향 도로에서 송파경...,단속 정보가 공개된 것에 대해 음주운전자들이 단속 지점을 피해 갈 우려가 있어 경찰...
8,조국 전 법무부 장관의 11가지 혐의가 56페이지에 걸쳐 서술된 검찰 공소장(지난달...,조 전 법무부 장관의 검찰 공소장에는 서울대 공익인권법센터가 인턴 증명서를 발급했다...
9,"미국과 이라크 내 친이란 세력, 이란 간의 갈등이 격화하면서 중동 정세가 격랑 속으...",이란을 배후로 지목한 미국이 보복 차원에서 카다이브 헤즈볼라의 군사시설을 폭격하자 ...


In [449]:
for i in next(iter(dl))[1]:
    prediction = ""
    for j in i:
        prediction += data.num_to_vocab[j]
    print(prediction)

<Sos>국민통합연대가연창립대회에자유한국당홍전대표등의원20여명을포함한500여명이자리를빼곡하게매웠으며축사는전목사가했다.
<Sos>아이엘사이언스대표송씨는발광다이오드조명용실리콘렌즈를세계최초로개발하여코스닥상장까지일궈냈다.<Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad>
<Sos>SK텔레콤은T끼리온가족할인으로요금을깎아주거나30년이상가입자를위해호텔코스요리와공연을마련하고가입자의이름을넣은헌정광고를제작했다.<Pad><Pad>
<Sos>대형마트3사가장바구니사용을독려하고플라스틱폐기물을줄이기위해포장용테이프와플라스틱끈을없앴는데소비자는불만을드러냈다.<Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad>
<Sos>현대차그룹은제네시스를이끌GV80의내·외관사진을공개하며이달중출시한다고밝혔다.<Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad>


In [425]:
for i in next(iter(dl))[2]:
    prediction = ""
    for j in i:
        prediction += data.num_to_vocab[j]
    print(prediction)

국민통합연대가연창립대회에자유한국당홍전대표등의원20여명을포함한500여명이자리를빼곡하게매웠으며축사는전목사가했다.<Eos>
아이엘사이언스대표송씨는발광다이오드조명용실리콘렌즈를세계최초로개발하여코스닥상장까지일궈냈다.<Eos><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad>
SK텔레콤은T끼리온가족할인으로요금을깎아주거나30년이상가입자를위해호텔코스요리와공연을마련하고가입자의이름을넣은헌정광고를제작했다.<Eos><Pad><Pad>
대형마트3사가장바구니사용을독려하고플라스틱폐기물을줄이기위해포장용테이프와플라스틱끈을없앴는데소비자는불만을드러냈다.<Eos><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad>
현대차그룹은제네시스를이끌GV80의내·외관사진을공개하며이달중출시한다고밝혔다.<Eos><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad><Pad>
